In [1]:
using DataFrames
using JSON
using Iterators
#using taxis
using HDF5, JLD
using Stats
using kNN
#using sequenceCompare
#reload("taxis")

nprocs()

8

In [2]:
println("Begin")

println("loading csv files")
taxi_df = readtable("/home/tony/ML/taxi/taxi2_time/train_100k.csv")
taxi_validation_df = readtable("/home/tony/ML/taxi/taxi2_time/test.csv")

println("loading coords")
taxi_df[:COORDS] = [float(hcat(JSON.parse(x)...)) for x in  taxi_df[:POLYLINE]]
taxi_validation_df[:COORDS] = [float(hcat(JSON.parse(x)...)) for x in taxi_validation_df[:POLYLINE]]

println("getting coords counts")
taxi_df[:NUM_COORDS] = [length(x)::Int64 for x in taxi_df[:COORDS]]
taxi_validation_df[:NUM_COORDS] = [length(x)::Int64 for x in taxi_validation_df[:COORDS]]

println("deleting unneeded data rows/columns")
delete!(taxi_validation_df, :POLYLINE)
delete!(taxi_df, :POLYLINE)

println("adding start/end point columns")
taxi_df[:START] = [x[:,1] for x in taxi_df[:COORDS]]
taxi_validation_df[:START] = [x[:,1] for x in taxi_validation_df[:COORDS]]

taxi_df[:END] = [x[:,end] for x in taxi_df[:COORDS]]
taxi_validation_df[:END] = [x[:,end] for x in taxi_validation_df[:COORDS]]

println("deleting training examples with no coords!")
#These examples are not going to be useful!
deleterows!(taxi_df, find(taxi_df[:NUM_COORDS] .== 0))

println("done!")

Begin
loading csv files
loading coords
getting coords counts
deleting unneeded data rows/columns
adding start/end point columns
deleting training examples with no coords!
done!


In [3]:
println("finding unique number of coords")
all_coords_val = hcat(taxi_validation_df[:COORDS]...)
all_coords = hcat(taxi_df[:COORDS]...)

finding unique number of coords


2x4774976 Array{Float64,2}:
 -8.61864  -8.6185  -8.62033  -8.62215  …  -8.6304  -8.63042  -8.63041
 41.1414   41.1414  41.1425   41.1438      41.1579  41.158    41.1579 

# Creating coord dict

In [4]:
#small_taxi_df = GetTableOrderedSubset(taxi_df, 10000)
#coordsDB = ConstructCoordsDatabase(small_taxi_df, 4)

#Creating new features


In [ ]:
function GetDateInfo(df)
    if haskey(df, :DAYOFWEEK)
        return df
end

function GetDistanceData(df)
    if haskey(df, :DISTANCE)
        return df
end

# Training models

In [4]:
# http://en.wikipedia.org/wiki/Dynamic_time_warping
function DTWDistance{T}(s::Array{T}, t::Array{T})
    n, m = size(s,2), size(t, 2)
    DTW = zeros(n+1,m+1)
    #println("sizeof DTW: ", size(DTW))
    DTW[:,1], DTW[1,:] = Inf, Inf
    DTW[1,1] = 0

    for i=2:n+1
        for j=2:m+1
            #println("i=", i, "/", n, " j=", j, "/", m, " size(s)=", size(s), " size(t)=", size(t))
            cost = sum((s[:,i-1] - t[:,j-1]).^2)

            DTW[i,j] = cost + min(DTW[i-1, j  ], #insertion
                                  DTW[i  , j-1], #deletion
                                  DTW[i-1, j-1]) #match
            #println("cost is ", cost, " i=", i, "/", n, " j=", j, "/", m, " DTW=", DTW[i,j])
        end
    end

    return DTW[n+1,m+1]
end

# note. there is also a windowed version for better performance!  See the wikipedia article


function GetAverageDistanceFromCoordsSequence(coords)
    #Outline of the basic algorithm
    #for coord_pair c_1..c_n
    # estimate <- for i=1:n average(15*(i-1) + average(for c_j in D[c+i] * exp(-|L_j-L_i|) * L_j)
    coords = round(Coords[i],round_len)       
        
        num_coords = size(coords,2)
end

function GetAverageDistancesByCoordsDict(TripIds, Coords, round_len=5)
    distaces = Array{Float64,0}()
    num_coords = size(TripIds,1)
    for i=1:num_coords
        trip_id = TripIds[i]

        coords = Coords[i]
        if length(coords) == 0
            continue
        end
        
        coords = round(Coords[i],round_len)       
        
        num_coords = size(coords,2)
        #println("num_coords: ", num_coords)
        for j=1:num_coords
            coord_pair = (coords[1,j], coords[2,j])
            #println("coord_pair: ", coord_pair, ", j/num_coords: ", j, "/", num_coords, ", thing: ", [SequenceRef(trip_id, j, num_coords-j)])
            if !haskey(D, coord_pair)
                D[coord_pair] = [SequenceRef(trip_id, j, num_coords-j)]
            else
                push!(D[coord_pair], SequenceRef(trip_id, j, num_coords-j))
            end        
        end
    end
    
    return D    
end

function findClosestTrainingExample(all_train_coords, test_path)
    num_paths = length(all_train_coords)
    best_dist = 9999.0
    best_path = all_train_coords[1]
    for k=1:num_paths
        train_path = all_train_coords[k]
        dist = DTWDistance(train_path, test_path) 
        #println("k=", k, " dist=", dist)
        if dist < best_dist
            #println(k, ", old best: ", best_dist, " new best: ", dist)
            best_dist = dist
            best_path = all_train_coords[k]
        end
    end
    
    return best_path
end

function findClosestTrainingExampleForTestSet(all_train_paths, all_test_paths)
    
    
    num_train_paths = length(all_train_paths)
    num_test_paths = length(all_test_paths)
    closest_examples = cell(num_test_paths)
    for k=1:num_test_paths
        if k % 20 == 0
            println(k, "/", num_test_paths, " for ", num_train_paths, " train path examples") 
        end
        test_path = all_test_paths[k]
        closest_training_example = findClosestTrainingExample(all_train_paths, test_path) 
        closest_examples[k] = closest_training_example
    end
    
    return closest_examples
end

function pFindClosestTrainingExampleForTestSet(all_train_paths, all_test_paths)
    function findClosestTrainingExample(all_train_coords, test_path)
        num_paths = length(all_train_coords)
        best_dist = 9999.0
        best_path = all_train_coords[1]
        for k=1:num_paths
            train_path = all_train_coords[k]
            dist = DTWDistance(train_path, test_path) 
            #println("k=", k, " dist=", dist)
            if dist < best_dist
                #println(k, ", old best: ", best_dist, " new best: ", dist)
                best_dist = dist
                best_path = all_train_coords[k]
            end
        end

        return best_path
    end
    
    getClosestExample = p -> findClosestTrainingExample(all_train_paths, p)
    return pmap(getClosestExample, all_test_paths)
end

pFindClosestTrainingExampleForTestSet (generic function with 1 method)

In [8]:
#@everywhere using taxis
#@everywhere using sequenceCompare
#@everywhere reload("taxis")
#@everywhere reload("sequenceCompare")
#reload("taxis")

all_train_coords = taxi_df[:COORDS][1:20000]
test_guess_paths = findClosestTrainingExampleForTestSet(all_train_coords, taxi_validation_df[:COORDS])

20/320 for 20000 train path examples
40/320 for 20000 train path examples
60/320 for 20000 train path examples
80/320 for 20000 train path examples
100/320 for 20000 train path examples
120/320 for 20000 train path examples
140/320 for 20000 train path examples
160/320 for 20000 train path examples
180/320 for 20000 train path examples
200/320 for 20000 train path examples
220/320 for 20000 train path examples
240/320 for 20000 train path examples
260/320 for 20000 train path examples
280/320 for 20000 train path examples
300/320 for 20000 train path examples
320/320 for 20000 train path examples


320-element Array{Any,1}:
 2x1 Array{Float64,2}:
 -8.58588
 41.1483                                                                                                                                
 2x18 Array{Float64,2}:
 -8.61094  -8.61045  -8.60924  -8.60837  …  -8.60393  -8.60395  -8.60395
 41.1457   41.1461   41.1468   41.1477      41.1615   41.1616   41.1616 
 2x7 Array{Float64,2}:
 -8.58016  -8.57873  -8.58034  -8.5828  -8.58244  -8.58103  -8.58287
 41.1471   41.1495   41.1523   41.1551  41.1587   41.1628   41.1661          
 2x3 Array{Float64,2}:
 -8.61555  -8.61535  -8.61453
 41.1407   41.1409   41.1412                                                                                        
 2x3 Array{Float64,2}:
 -8.61981  -8.61979  -8.61978
 41.148    41.1481   41.1481                                                                                        
 2x4 Array{Float64,2}:
 -8.62776  -8.62755  -8.62635  -8.6259
 41.1754   41.1748   41.1723   41.1704                        

In [ ]:
taxi_validation_df[:GUESS_PATHS] = test_guess_paths

In [ ]:
#guess_times = [length(x)*15 for x in test_guess_paths]
num_test_examples = length(test_guess_paths)

guess_times = Array(Int64,num_test_examples)
dest_coords = cell(num_test_examples)

all_test_paths = taxi_validation_df[:COORDS]
for k=1:num_test_examples
    test_path = all_test_paths[k]
    best_guess_path = test_guess_paths[k]
    
    test_path_time = length(test_path)*15
    best_guess_time = length(best_guess_path)*15
    
    if length(test_path) > length(best_guess_path)
        println(k, ":  guessing ", best_guess_time, " but existing time is ", test_path_time)
        best_guess_time = max(660, test_path_time)
    end
    guess_times[k] = best_guess_time
end

submission_validation = guess_times

#Generating Submission

In [ ]:
#mean_time = mean(times_validation)
#submission_validation = [max(x, mean_time) for x in times_validation]

df_submission = DataFrame()
df_submission[:TRIP_ID] = taxi_validation_df[:TRIP_ID]
df_submission[:TRAVEL_TIME] = submission_validation
writetable("third_submission_50k.csv", df_submission)

In [388]:
immutable Point2{T}
  x::T
  y::T
end

D = [Point2(1.,2.) => 42]
haskey(D, Point2(1., 2.))  #False!


true

LoadError: ArgumentError("setindex!(::DataFrame, ...) only broadcasts scalars, not arrays")
while loading In[42], in expression starting on line 2

In [39]:
taxi_validation_df[:COORDS]

320-element DataArray{Any,1}:
 2x11 Array{Float64,2}:
 -8.58568  -8.58571  -8.58568  -8.58573  …  -8.587   -8.58658  -8.58488
 41.1485   41.1486   41.1489   41.1489      41.1475  41.1472   41.1466    
 2x40 Array{Float64,2}:
 -8.61088  -8.61086  -8.6109  -8.61044  …  -8.60293  -8.60255  -8.60189
 41.1456   41.1456   41.1458  41.1462      41.1628   41.1631   41.1636    
 2x40 Array{Float64,2}:
 -8.58574  -8.58573  -8.58572  -8.58629  …  -8.57695  -8.5759  -8.5749
 41.1486   41.1488   41.149    41.149       41.1664   41.1672  41.1677     
 2x8 Array{Float64,2}:
 -8.61396  -8.61412  -8.61509  -8.61528  …  -8.61524  -8.61505  -8.61464
 41.1412   41.1411   41.1409   41.1408      41.1408   41.1408   41.141    
 2x2 Array{Float64,2}:
 -8.6199  -8.61989
 41.148   41.148                                                                                                                
 2x137 Array{Float64,2}:
 -8.63061  -8.63061  -8.63074  -8.63151  …  -8.62639  -8.6264  -8.62641
 41.1782   41.1782